In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

#import my modules
import acquire 
import wrangle

Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.

In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.



Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [3]:
zillow_df= acquire.get_new_zillow()

In [4]:
zillow_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [5]:
zillow_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 115.0 MB


Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaninful; remember to document your process and decisions using markdown and code commenting where helpful.

In [6]:
def miss_dup_values(df):
    '''
    this function takes a dataframe as input and will output metrics for missing values and duplicated rows, 
    and the percent of that column that has missing values and duplicated rows
    '''
        # Total missing values
    mis_val = df.isnull().sum()
        # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
        #total of duplicated
    dup = df.duplicated().sum()  
        # Percentage of missing values
    dup_percent = 100 * dup / len(df)
        # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
    mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
        # Print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
           "There are " + str(mis_val_table_ren_columns.shape[0]) +
           " columns that have missing values.")
    print( "  ")
    print (f"** There are {dup} duplicate rows that represents {round(dup_percent, 2)}% of total Values**")
        # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [7]:
#using the function above
miss_dup_values(zillow_df)

Your selected dataframe has 7 columns.
There are 6 columns that have missing values.
  
** There are 11644 duplicate rows that represents 0.54% of total Values**


,Missing Values,% of Total Values
yearbuilt,9337,0.4
calculatedfinishedsquarefeet,8484,0.4
taxamount,4442,0.2
taxvaluedollarcnt,493,0.0
bedroomcnt,11,0.0
bathroomcnt,11,0.0


In [8]:
zillow_df['bedroomcnt'].nunique()

19

In [9]:
zillow_df['bedroomcnt'].value_counts().sort_index()

0.0      13187
1.0      23166
2.0     335473
3.0     964298
4.0     634289
5.0     150866
6.0      25166
7.0       4807
8.0       1107
9.0        291
10.0       121
11.0        34
12.0        12
13.0        16
14.0         7
15.0         6
16.0         2
18.0         3
25.0         1
Name: bedroomcnt, dtype: int64

In [10]:
# drop duplicates
zillow_df = zillow_df.drop_duplicates()
zillow_df.shape

(2141219, 7)

In [11]:
#drop nulls
zillow_df = zillow_df.dropna(how='any',axis=0)

In [13]:
#size after the drops null
zillow_df.shape

(2130214, 7)

In [14]:
#confirm that we dond have nulls
zillow_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2130214 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2130214 non-null  float64
 1   bathroomcnt                   2130214 non-null  float64
 2   calculatedfinishedsquarefeet  2130214 non-null  float64
 3   taxvaluedollarcnt             2130214 non-null  float64
 4   yearbuilt                     2130214 non-null  float64
 5   taxamount                     2130214 non-null  float64
 6   fips                          2130214 non-null  float64
dtypes: float64(7)
memory usage: 130.0 MB


In [15]:
cols = zillow_df.columns.to_list()
for col in cols:
    print(col)
    print(zillow_df[col].value_counts().sort_index().head(26))

bedroomcnt
0.0       4259
1.0      22713
2.0     333208
3.0     958078
4.0     630289
5.0     150194
6.0      25092
7.0       4791
8.0       1103
9.0        290
10.0       118
11.0        34
12.0        12
13.0        15
14.0         7
15.0         5
16.0         2
18.0         3
25.0         1
Name: bedroomcnt, dtype: int64
bathroomcnt
0.00       4199
0.50         16
1.00     411264
1.50      30275
1.75          3
2.00     938213
2.50     141089
3.00     421066
3.50      28286
4.00      81903
4.50      19396
5.00      28299
5.50       6174
6.00      10717
6.50       1330
7.00       4381
7.50        382
8.00       1681
8.50        108
9.00        707
9.50         50
10.00       322
10.50        14
11.00       145
11.50         3
12.00        73
Name: bathroomcnt, dtype: int64
calculatedfinishedsquarefeet
1.0      6
2.0      2
3.0      7
7.0      1
10.0     2
12.0     1
20.0    15
30.0     3
31.0     1
32.0     1
40.0     9
43.0     1
48.0     2
54.0     1
56.0     1
60.0     8
63.0    

In [16]:
#bedrooms, yearbuilt and fips can be converted to int
zillow_df[['bedroomcnt', 'yearbuilt', 'fips']] = zillow_df[['bedroomcnt', 'yearbuilt', 'fips']].astype(int)

In [19]:
#a house with more that one bedroom and has less than 10 squarefeet????
zillow_df[zillow_df.calculatedfinishedsquarefeet < 10]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
58438,2,1.0,1.0,121376.0,1907,1996.35,6037
122963,2,2.0,3.0,584804.0,1955,6822.61,6037
402516,1,4.0,3.0,356717.0,1986,3840.63,6037
584383,5,2.0,3.0,139441.0,1958,1676.86,6037
587783,4,3.0,2.0,1219502.0,1952,14299.74,6037
683031,2,3.0,3.0,981300.0,1950,10657.81,6037
724785,5,7.0,7.0,1653839.0,1925,20140.20,6037
821592,1,4.0,3.0,1827450.0,1991,22011.62,6037
1046787,0,0.0,1.0,28091.0,1963,439.55,6037
1276353,1,3.0,1.0,124906.0,1953,2020.66,6037


In [20]:
zillow_df[zillow_df.calculatedfinishedsquarefeet < 500].count()

bedroomcnt                      4736
bathroomcnt                     4736
calculatedfinishedsquarefeet    4736
taxvaluedollarcnt               4736
yearbuilt                       4736
taxamount                       4736
fips                            4736
dtype: int64

In [21]:
zillow_df[(zillow_df.bedroomcnt == 0) & (zillow_df.bathroomcnt == 0)].count()

bedroomcnt                      3271
bathroomcnt                     3271
calculatedfinishedsquarefeet    3271
taxvaluedollarcnt               3271
yearbuilt                       3271
taxamount                       3271
fips                            3271
dtype: int64

In [22]:
#get the index to drop the rows
ind = list(zillow_df[(zillow_df.bedroomcnt == 0) & (zillow_df.bathroomcnt == 0)].index)

In [24]:
3271 / zillow_df.shape[0]

0.0015355264776214972

In [25]:
#drop
zillow_df.drop(ind, axis=0, inplace= True)

In [26]:
zillow_df['fips'].value_counts()

6037    1419556
6059     546326
6111     161061
Name: fips, dtype: int64

Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe witn no missing values in your wrangle.py file. Name your final function wrangle_zillow.

In [27]:
def clean_zillow (df):
    '''
    Takes in a df and drops duplicates,  nulls, all houses that do not have bedrooms and bathrooms,
    houses that calculatedfinishedsquarefeet < 800, and bedroomcnt, yearbuilt, fips are changed to
    int.
    Return a clean df
    '''
    
    # drop duplicates
    df = df.drop_duplicates()
    #drop nulls
    df = df.dropna(how='any',axis=0)

    #drop all houses with bath = 0 and bedromms = 0
    #get the index to drop the rows
    ind = list(df[(df.bedroomcnt == 0) & (df.bathroomcnt == 0)].index)
    #drop
    df.drop(ind, axis=0, inplace= True)


    #drop all houses calculatedfinisheedsqf <800
    #get the index to drop
    lis =list(df[df['calculatedfinishedsquarefeet'] < 800].index)
    #drop the rows
    df.drop(lis, axis=0, inplace = True)

    #bedrooms, yearbuilt and fips can be converted to int
    df[['bedroomcnt', 'yearbuilt', 'fips']] = df[['bedroomcnt', 'yearbuilt', 'fips']].astype(int)
    return df


In [28]:
df = wrangle.wrangle_zillow()

In [30]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4,2.0,3633.0,296425.0,2005,6941.39,6037
6,3,4.0,1620.0,847770.0,2011,10244.94,6037
7,3,2.0,2077.0,646760.0,1926,7924.68,6037
18,3,1.0,1244.0,169471.0,1950,2532.88,6037
19,3,2.0,1300.0,233266.0,1950,3110.99,6037


In [32]:
#Regular .info() wont show null count because larger data like zillow behave like that
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2074798 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2074798 non-null  int64  
 1   bathroomcnt                   2074798 non-null  float64
 2   calculatedfinishedsquarefeet  2074798 non-null  float64
 3   taxvaluedollarcnt             2074798 non-null  float64
 4   yearbuilt                     2074798 non-null  int64  
 5   taxamount                     2074798 non-null  float64
 6   fips                          2074798 non-null  int64  
dtypes: float64(4), int64(3)
memory usage: 126.6 MB
